## Prepare test data across four domains

In [ ]:
# All obtained from the MHH data using fetch_test_data.sql
import pandas as pd
import numpy as np
from pathlib import Path

np.random.seed(42)

file_paths = [
    Path("../data/mhh_condition.csv"),
    Path("../data/mhh_procedure.csv"),
    Path("../data/mhh_medication.csv"),
    Path("../data/mhh_measurement.csv"),
]

for file_path in file_paths:
    
    print(f"Processing {file_path.stem.split('_')[1]}")
    df = pd.read_csv(file_path)

    # Rename first column to keyword
    df.rename(columns={df.columns[0]: "keyword", df.columns[1]: "count"}, inplace=True)
    
    df = df[df["keyword"].notna()]
    df = df[df["keyword"].str.strip() != ""]

    if file_path.stem.split('_')[1] == "medication":
        df["omop_table"] = "drug_exposure"
        df["omop_field"] = "drug_concept_id"
    elif file_path.stem.split('_')[1] == "condition":
        df["omop_table"] = "condition_occurrence"
        df["omop_field"] = "condition_concept_id"
    elif file_path.stem.split('_')[1] == "procedure":
        df["omop_table"] = "procedure_occurrence"
        df["omop_field"] = "procedure_concept_id"
    elif file_path.stem.split('_')[1] == "measurement":
        df["omop_table"] = "measurement"
        df["omop_field"] = "measurement_concept_id"

    # Eliminate the lowest 5% by count
    lower_5pct_cutoff = df["count"].quantile(0.05)
    df = df[df["count"] > lower_5pct_cutoff]

    df = df.sort_values("count", ascending=False).reset_index(drop=True)

    # Now apply quartile sampling as before
    total_samples = 200  # Total samples we want
    sampled_list = []
    
    unique_counts = df["count"].nunique()
    df["quartile"] = pd.qcut(df["count"], q=4, labels=["Q4", "Q3", "Q2", "Q1"], duplicates="drop")
    for q in ["Q4", "Q3", "Q2", "Q1"]:
        quartile_data = df[df["quartile"] == q]
        sample_size = min(50, len(quartile_data))
        if sample_size > 0:
            sampled = quartile_data.sample(n=sample_size, random_state=42)
            sampled_list.append(sampled)

    sampled_list = [df.sample(n=min(total_samples, len(df)), random_state=42)]
    result = pd.concat(sampled_list, ignore_index=True).sort_values("count", ascending=False).reset_index(drop=True)
    result.to_csv(f"../data/evaluate/{file_path.stem.split('_')[1]}_mapping.csv", index=False)